In [ ]:
from transformers import pipeline

# Load a GPT-like model that supports few-shot learning (or use a similar model)
llm = pipeline('text-generation', model="gpt-3")

# Prepare the few-shot prompt
few_shot_prompt = """
Extract the sentiment expression for each aspect mentioned in the review. Below are a few examples:

Review: "The product quality is excellent but the customer service is terrible."
Aspects: 
- quality: excellent
- customer service: terrible

Review: "The battery life of this laptop is too short, but the screen quality is amazing."
Aspects:
- battery life: too short
- screen quality: amazing

Review: "The quality is good but the price is expensive."
Aspects:
- quality: good
- price: expensive
"""

# Input the review for inference
input_review = "The quality is good but the price is expensive."

# Generate the output using the LLM
output = llm(few_shot_prompt, max_length=200)

# Print the output
print(output)


# Llama 3.1 to extract sentiment expression
- unable to restrict / standardize the output
- using few shots : model include the examples in the output too

In [2]:
import ollama

list_model_name = ['llama3.1']

response = ollama.chat(model='llama3.1', messages=[
  {
    'role': 'user',
    'content': '''

Extract the sentiment expression for each aspect mentioned in the review. Below are a few examples:

Review: "The product quality is excellent but the customer service is terrible."
Aspects: 
- quality: excellent
- customer service: terrible

Review: "The battery life of this laptop is too short, but the screen quality is amazing."
Aspects:
- battery life: too short
- screen quality: amazing

Review: "The quality is good but the price is expensive."
Aspects:
- quality: good
- price: expensive
    ''',
  },
])
print(response['message']['content'])

Let's apply the same process to a new review:

Review: "I'm really satisfied with the hotel's cleanliness and location, but the breakfast was a letdown."

Aspects:
- cleanliness: really satisfied (positive)
- location: really satisfied (positive)
- breakfast: a letdown (negative)

Example 2:
Review: "The new smartphone has a great camera and decent battery life, but the storage capacity is too small."
 
Aspects:
- camera: great
- battery life: decent
- storage capacity: too small


In [25]:
import ollama

# Define a function to interact with Llama3.1 via Ollama
def extract_sentiment_expressions(model_name, review, aspects):
    # Construct the prompt dynamically with the review and aspects
    prompt = f'''
Extract the sentiment expression for each aspect mentioned in the review. Return the output in this json format below. Return the json only, no other text 

  "review": "The new smartphone has a great camera and decent battery life, but the storage capacity is too small.",
  "aspects": 
    "Battery Life": "decent",
    "Storage": "too small"


Review: "{review}"
Aspects:
'''

    # Add the aspects dynamically to the prompt
    for aspect in aspects:
        prompt += f"- {aspect}: \n"

    # Use the Ollama API to generate the sentiment expression
    response = ollama.chat(
        model=model_name, 
        messages=[
            {
                'role': 'user',
                'content': prompt,
            }
        ]
    )

    # Return the response from the model
    return response['message']['content']

# Example usage
model_name = 'llama3.1'
review = "The quality is good but the price is expensive."
aspects = ['quality', 'price']

# Call the function to extract sentiment expressions
sentiment_expressions = extract_sentiment_expressions(model_name, review, aspects)
print(sentiment_expressions)


```
{
  "review": "The new smartphone has a great camera and decent battery life, but the storage capacity is too small.",
  "aspects": [
    {
      "aspect": "Battery Life",
      "sentiment_expression": "decent"
    },
    {
      "aspect": "Storage",
      "sentiment_expression": "too small"
    }
  ]
}
```

```
{
  "review": "The quality is good but the price is expensive.",
  "aspects": [
    {
      "aspect": "quality",
      "sentiment_expression": "good"
    },
    {
      "aspect": "price",
      "sentiment_expression": "expensive"
    }
  ]
}
```


In [27]:
# Example usage
model_name = 'llama3.1'
review = '''
I placed my order for shipping but apparently the warehouse was out of the black ink so they switched to store delivery to ship the color cartridge. When I called customer support I was issued a refund but was also promised an e-mail gift card to cover the reorder since I would now be charged shipping fee. I was placed on hold so she could issue the refund and send me the gift card but somehow was transferred to a different person who transferred me back to customer service only to get someone different who looked in my account, told me that the original person had placed the item back in my cart for me and selected shipping and that the gift card can take a bit to arrive in my email but gave me a promo code for all the confusion which I di...
'''
aspects = ['Delivery']

# Call the function to extract sentiment expressions
sentiment_expressions = extract_sentiment_expressions(model_name, review, aspects)
print(sentiment_expressions)

{
  "review": "The new smartphone has a great camera and decent battery life, but the storage capacity is too small.",
  "aspects": [
    {
      "name": "Battery Life",
      "sentiment_expression": "decent"
    },
    {
      "name": "Storage",
      "sentiment_expression": "too small"
    }
  ],
  "Review": "I placed my order for shipping but apparently the warehouse was out of the black ink so they switched to store delivery to ship the color cartridge. When I called customer support I was issued a refund but was also promised an e-mail gift card to cover the reorder since I would now be charged shipping fee. I was placed on hold so she could issue the refund and send me the gift card but somehow was transferred to a different person who transferred me back to customer service only to get someone different who looked in my account, told me that the original person had placed the item back in my cart for me and selected shipping and that the gift card can take a bit to arrive in my 

In [30]:
# Example usage
model_name = 'llama3.1'
review = '''
Fast ship good price. It's a shame I had to order this from another walmart and not my local store.
'''

aspects = ['Price', 'Delivery']

# Call the function to extract sentiment expressions
sentiment_expressions = extract_sentiment_expressions(model_name, review, aspects)
print(sentiment_expressions)

{
  "review": "Fast ship good price. It's a shame I had to order this from another walmart and not my local store.",
  "aspects": [
    {
      "aspect": "Price",
      "sentiment_expression": "good"
    },
    {
      "aspect": "Delivery",
      "sentiment_expression": "Fast ship"
    }
  ]
}


# Llama 3.1 - Q&A style to extract sentiment expressions and loop thru all aspects identified
- output not great, as unable to correctly extract the sentiment expressions

In [49]:
import ollama

def extract_sentiment_expression_llama(review, aspects, model_name='llama3.1'):
    # Store the answers
    answers = {}

    # Iterate over the provided aspects to construct the prompt
    for aspect in aspects:
        prompt = f"""
        Review: "{review}"
        Aspect: "{aspect}"
        What is the sentiment expression for this aspect? Return the sentiment expression identified only. Example of sentiment expressions are 'good', 'too late'
        """

        # Use the Ollama API to generate the sentiment expression
        response = ollama.chat(
            model=model_name, 
            messages=[{"role": "user", "content": prompt}]
        )

        # Extract the sentiment expression from the response
        result_text = response['message']['content']
        answers[aspect] = result_text.strip()

    return answers

# Example usage
review = "The quality is good but the price is expensive."
aspects = ["quality", "price"]

# Extract sentiment expressions for the review and specified aspects
sentiment_expressions = extract_sentiment_expression_llama(review, aspects)
print(f"Review: '{review}' => Sentiment Expressions: {sentiment_expressions}")


Review: 'The quality is good but the price is expensive.' => Sentiment Expressions: {'quality': 'good', 'price': '"expensive"'}


In [51]:
# Example usage
review = "Fast ship good price. It's a shame I had to order this from another walmart and not my local store."
aspects = ['Price', 'Delivery']

# Extract sentiment expressions for the review and specified aspects
sentiment_expressions = extract_sentiment_expression_llama(review, aspects)
print(f"Review: '{review}' => Sentiment Expressions: {sentiment_expressions}")


Review: 'Fast ship good price. It's a shame I had to order this from another walmart and not my local store.' => Sentiment Expressions: {'Price': 'Good', 'Delivery': 'Good'}


In [52]:
# Example usage
review = "I am not satisfied with the HP instant Ink service for both my Tango printer and the HP Envy 6000. In fact, I did not receive ink cartridges when I needed them. I canceled both HP instant Ink subscriptions and I am still being harassed about these subscriptions even though they have been deleted."
aspects = ['Instant Ink', 'Delivery']

# Extract sentiment expressions for the review and specified aspects
sentiment_expressions = extract_sentiment_expression_llama(review, aspects)
print(f"Review: '{review}' => Sentiment Expressions: {sentiment_expressions}")


Review: 'I am not satisfied with the HP instant Ink service for both my Tango printer and the HP Envy 6000. In fact, I did not receive ink cartridges when I needed them. I canceled both HP instant Ink subscriptions and I am still being harassed about these subscriptions even though they have been deleted.' => Sentiment Expressions: {'Instant Ink': 'Frustration/anger', 'Delivery': '"Too late"'}


# (BEST) Llama 3.1 - using aspects identified, perform sentiment analysis directly
 - Good performance
 - Consistent output
 - Simplified pipeline

In [3]:
import ollama

def extract_sentiment_expression_llama(review, aspects, model_name='llama3.1'):
    # Store the answers
    answers = {}

    # Iterate over the provided aspects to construct the prompt
    for aspect in aspects:
        prompt = f"""
        Review: "{review}"
        Aspect: "{aspect}"
        What is the sentiment (positive, negative) for this aspect? Return the sentiment identified only. 
        """

        # Use the Ollama API to generate the sentiment expression
        response = ollama.chat(
            model=model_name, 
            messages=[{"role": "user", "content": prompt}]
        )

        # Extract the sentiment expression from the response
        result_text = response['message']['content']
        answers[aspect] = result_text.strip()

    return answers

# Example usage
review = "The quality is good but the price is expensive."
aspects = ["quality", "price"]

# Extract sentiment expressions for the review and specified aspects
sentiment_expressions = extract_sentiment_expression_llama(review, aspects)
print(f"Review: '{review}' => Sentiment Expressions: {sentiment_expressions}")


Review: 'The quality is good but the price is expensive.' => Sentiment Expressions: {'quality': 'Neutral/Positive', 'price': 'Negative'}


In [55]:
# Example usage
review = "I am not satisfied with the HP instant Ink service for both my Tango printer and the HP Envy 6000. In fact, I did not receive ink cartridges when I needed them. I canceled both HP instant Ink subscriptions and I am still being harassed about these subscriptions even though they have been deleted."
aspects = ['Instant Ink', 'Delivery']

# Extract sentiment expressions for the review and specified aspects
sentiment_expressions = extract_sentiment_expression_llama(review, aspects)
print(f"Review: '{review}' => Sentiment Expressions: {sentiment_expressions}")


Review: 'I am not satisfied with the HP instant Ink service for both my Tango printer and the HP Envy 6000. In fact, I did not receive ink cartridges when I needed them. I canceled both HP instant Ink subscriptions and I am still being harassed about these subscriptions even though they have been deleted.' => Sentiment Expressions: {'Instant Ink': 'Negative', 'Delivery': 'Negative'}


In [56]:
# Example usage
review = "Fast ship good price. It's a shame I had to order this from another walmart and not my local store."
aspects = ['Price', 'Delivery']

# Extract sentiment expressions for the review and specified aspects
sentiment_expressions = extract_sentiment_expression_llama(review, aspects)
print(f"Review: '{review}' => Sentiment Expressions: {sentiment_expressions}")


Review: 'Fast ship good price. It's a shame I had to order this from another walmart and not my local store.' => Sentiment Expressions: {'Price': 'Positive', 'Delivery': 'Positive'}


In [57]:
# Example usage
review = "Have had a lot of difficulty with the printer. Instant ink was terrible and cost me more money due to limiting my printing. Cartridges are expensive and don't seem to last very long."
aspects = ['Price', 'Instant Ink', 'Product Quality']

# Extract sentiment expressions for the review and specified aspects
sentiment_expressions = extract_sentiment_expression_llama(review, aspects)
print(f"Review: '{review}' => Sentiment Expressions: {sentiment_expressions}")


Review: 'Have had a lot of difficulty with the printer. Instant ink was terrible and cost me more money due to limiting my printing. Cartridges are expensive and don't seem to last very long.' => Sentiment Expressions: {'Price': 'Negative', 'Instant Ink': 'Negative', 'Product Quality': 'Negative'}


In [58]:
# Example usage
review = "Joined instant ink and you let me run out of color ink I have to buy at store while waiting for shipment"
aspects = ['Instant Ink', 'Delivery']

# Extract sentiment expressions for the review and specified aspects
sentiment_expressions = extract_sentiment_expression_llama(review, aspects)
print(f"Review: '{review}' => Sentiment Expressions: {sentiment_expressions}")


Review: 'Joined instant ink and you let me run out of color ink I have to buy at store while waiting for shipment' => Sentiment Expressions: {'Instant Ink': 'Negative', 'Delivery': 'Negative'}


In [59]:
# Example usage
review = '''
The printer is fine, but the excessive increase in the price of instant ink would make me suggest anyone looking 
for a new printer looks at alternatives first and does the calculation on relative costs.  
The setup for me was ok, but for many it is a bit convoluted.'''

aspects = ['Product Quality', 'Price']

# Extract sentiment expressions for the review and specified aspects
sentiment_expressions = extract_sentiment_expression_llama(review, aspects)
print(f"Review: '{review}' => Sentiment Expressions: {sentiment_expressions}")


Review: '
The printer is fine, but the excessive increase in the price of instant ink would make me suggest anyone looking 
for a new printer looks at alternatives first and does the calculation on relative costs.  
The setup for me was ok, but for many it is a bit convoluted.' => Sentiment Expressions: {'Product Quality': 'Neutral/Negative', 'Price': 'Negative'}


In [4]:
import pandas as pd

# Read the Excel file
file_path = 'predicted_labels_v2.xlsx'
df = pd.read_excel(file_path)
df.head()

,Survey ID,Product Name,Print Customer Region,LTR,Source Type,Survey language,Review Source,Star Rating,Product Family,Supplies Family,...,max_predicted_level1,max_predicted_aspect,All Aspects,Aspect 1,Aspect 2,Aspect 3,Unnamed: 25,Output Labels,Label Vectors,Predicted Labels
0,101618952,HP 63 Black Original Ink Cartridge,US,10,Web Reviews,English,Walmart,5.0,Supplies - Ink,Dolmen Refresh,...,Brand Loyalty,Others,"Price, Product Quality,",Price,Product Quality,NaN,NaN,"['Price', 'Product Quality']","[0, 1, 1, 0, 0]","['Product Quality', 'Price']"
1,110317001,HP 63XL High Yield Black Original Ink Cartridge,US,10,Web Reviews,English,HP US,5.0,Supplies - Ink,Dolmen Refresh,...,Brand Loyalty,Others,"Delivery, ,",Delivery,NaN,NaN,NaN,['Delivery'],"[1, 0, 0, 0, 0]",['Delivery']
2,113652950,HP 63 Black Original Ink Cartridge,US,10,Web Reviews,English,Walmart,5.0,Supplies - Ink,Dolmen Refresh,...,Brand Loyalty,Others,"Price, ,",Price,NaN,NaN,NaN,['Price'],"[0, 0, 1, 0, 0]",['Price']
3,117483634,HP 65XL Tri-color Original Ink Cartridge,US,10,Web Reviews,English,Walmart,5.0,Supplies - Ink,Dolmen Refresh,...,Brand Loyalty,Others,"Price, Product Quality,",Price,Product Quality,NaN,NaN,"['Price', 'Product Quality']","[0, 1, 1, 0, 0]",['Price']
4,118156397,HP 951XL 3-pack High Yield Cyan/Magenta/Yellow...,US,10,Web Reviews,English,Walmart,5.0,Supplies - Ink,Nesta+,...,Brand Loyalty,Others,"Price, Delivery,",Price,Delivery,NaN,NaN,"['Price', 'Delivery']","[1, 0, 1, 0, 0]","['Delivery', 'Price']"


In [5]:
import ast

# Function to convert 'Predicted Labels' column to a list
def convert_to_list(aspect_string):
    return ast.literal_eval(aspect_string) if isinstance(aspect_string, str) else []

# Applying the function to the DataFrame
def process_dataframe(df):
    # Create a new column for the list of aspects
    df['Aspect List'] = df['Predicted Labels'].apply(convert_to_list)

    # Apply the sentiment extraction function to each row
    df['Sentiment Expressions'] = df.apply(
        lambda row: extract_sentiment_expression_llama(row['Combined Text'], row['Aspect List']), axis=1
    )

    return df

# apply to df
df_new = process_dataframe(df)

# Save the new DataFrame to an Excel file
df_new.to_excel('processed_sentiment_data_v2.xlsx', index=False)

print("Data saved to 'processed_sentiment_data_v2.xlsx'")

Data saved to 'processed_sentiment_data_v2.xlsx'


## BertForSequenceClassification - predict sentiment for each aspects identified
- performance no good

In [31]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification

# Load the BERT model and tokenizer
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name)

# Define a function to extract sentiment for a review
def extract_sentiment(review, aspects):
    # Prepare input for the model
    input_texts = [f'{aspect}: {review}' for aspect in aspects]
    inputs = tokenizer(input_texts, return_tensors='pt', padding=True, truncation=True)
    
    # Perform inference
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Get predictions
    predictions = torch.argmax(outputs.logits, dim=-1).tolist()

    # Convert predictions to sentiment labels
    sentiment_labels = ['negative', 'neutral', 'positive']
    results = []

    for aspect, pred in zip(aspects, predictions):
        sentiment = sentiment_labels[pred]
        results.append(f'{review}, [{aspect}] [sentiment: {sentiment}]')

    return results

# Example usage
review = "The quality is good but the price is expensive."
aspects = ["quality", "price"]

# Extract sentiments for the given review and aspects
sentiment_results = extract_sentiment(review, aspects)
for result in sentiment_results:
    print(result)


A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

The quality is good but the price is expensive., [quality] [sentiment: negative]
The quality is good but the price is expensive., [price] [sentiment: negative]


In [32]:
# Example usage
review = "Fast ship good price. It's a shame I had to order this from another walmart and not my local store."
aspects = ['Price', 'Delivery']

# Extract sentiments for the given review and aspects
sentiment_results = extract_sentiment(review, aspects)
for result in sentiment_results:
    print(result)

Fast ship good price. It's a shame I had to order this from another walmart and not my local store., [Price] [sentiment: negative]
Fast ship good price. It's a shame I had to order this from another walmart and not my local store., [Delivery] [sentiment: negative]


# Distilbert for QA

In [60]:
import torch
from transformers import DistilBertTokenizer, DistilBertForQuestionAnswering

# Load the DistilBERT tokenizer and model
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertForQuestionAnswering.from_pretrained('distilbert-base-uncased')

# Function to extract sentiment expression using prompt engineering
def extract_sentiment_expression(review, aspect):
    # Construct the question based on the aspect
    question = f"What is the sentiment expression for the aspect '{aspect}'?"
    
    # Tokenize the input (question and review)
    inputs = tokenizer(question, review, return_tensors='pt', truncation=True)

    # Move tensors to the GPU if available
    if torch.cuda.is_available():
        inputs = {key: val.to('cuda') for key, val in inputs.items()}
        model.to('cuda')

    # Get model predictions
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Get the start and end logits for the answer
    start_logits = outputs.start_logits
    end_logits = outputs.end_logits

    # Get the predicted start and end token indices
    start_index = torch.argmax(start_logits)
    end_index = torch.argmax(end_logits) + 1  # +1 because we want to include the end token

    # Convert tokens to words
    tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])
    
    # Extract the answer based on the predicted indices
    answer = tokens[start_index:end_index]
    
    # Convert token list back to string
    sentiment_expression = tokenizer.convert_tokens_to_string(answer)

    return sentiment_expression.strip()

# Example usage
review = "The quality is good but the price is expensive."
aspect = "quality"

# Extract sentiment expression for the specified aspect
sentiment_expression = extract_sentiment_expression(review, aspect)
print(f"Sentiment expression for '{aspect}': {sentiment_expression}")

aspect = "price"
sentiment_expression = extract_sentiment_expression(review, aspect)
print(f"Sentiment expression for '{aspect}': {sentiment_expression}")


Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Sentiment expression for 'quality': 
Sentiment expression for 'price': [SEP]


# Q&A 
- several not found
- if sentence too long, will fail

In [4]:
from transformers import pipeline
import torch

# Check if GPU is available and set the device accordingly
device = 0 if torch.cuda.is_available() else -1

# Load a pre-trained question-answering model with GPU support if available
qa_pipeline = pipeline("question-answering", device=device)

def extract_sentiment_expression_qa(review, aspects):
    # Store the answers
    answers = {}

    # Iterate over the provided aspects to construct questions
    for aspect in aspects:
        question = f"What is the sentiment expression for the aspect '{aspect}'?"
        
        # Use the QA pipeline to find the answer to the question
        result = qa_pipeline(question=question, context=review)
        
        # Store the answer if the confidence score is above a threshold
        answers[aspect] = result['answer'] if result['score'] > 0.5 else "Not found"

    return answers

# Example reviews and aspects
review = "The quality is good but the price is expensive."
aspects = ["quality", "price"]

# Extract sentiment expressions for the review and specified aspects
sentiment_expressions = extract_sentiment_expression_qa(review, aspects)
print(f"Review: '{review}' => Sentiment Expressions: {sentiment_expressions}")


No model was supplied, defaulted to distilbert/distilbert-base-cased-distilled-squad and revision 626af31 (https://huggingface.co/distilbert/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.


Review: 'The quality is good but the price is expensive.' => Sentiment Expressions: {'quality': 'good', 'price': 'expensive'}


In [5]:
# if using 0.5 - both not found
# if using 0.2 - wrong sentiment
# # Example reviews and aspects
review = "Fast ship good price. It's a shame I had to order this from another walmart and not my local store"
aspects = ['Price', 'Delivery']

# Extract sentiment expressions for the review and specified aspects
sentiment_expressions = extract_sentiment_expression_qa(review, aspects)
print(f"Review: '{review}' => Sentiment Expressions: {sentiment_expressions}")


Review: 'Fast ship good price. It's a shame I had to order this from another walmart and not my local store' => Sentiment Expressions: {'Price': 'shame', 'Delivery': 'a shame'}


In [44]:
# Example reviews and aspects
review = "Fast ship good price. It's a shame I had to order this from another walmart and not my local store"
aspects = ['Price', 'Ship']

# Extract sentiment expressions for the review and specified aspects
sentiment_expressions = extract_sentiment_expression_qa(review, aspects)
print(f"Review: '{review}' => Sentiment Expressions: {sentiment_expressions}")


Review: 'Fast ship good price. It's a shame I had to order this from another walmart and not my local store' => Sentiment Expressions: {'Price': 'Not found', 'Ship': 'Not found'}


In [41]:
# Example reviews and aspects
review = "I have always been able to get my ink from Walmart even when other places have been out of stock .  The price has gone up a little bit but still the best price in town for ink cartridge."
aspects = ['Price']

# Extract sentiment expressions for the review and specified aspects
sentiment_expressions = extract_sentiment_expression_qa(review, aspects)
print(f"Review: '{review}' => Sentiment Expressions: {sentiment_expressions}")


Review: 'I have always been able to get my ink from Walmart even when other places have been out of stock .  The price has gone up a little bit but still the best price in town for ink cartridge.' => Sentiment Expressions: {'Price': 'Not found'}


In [45]:
# Example reviews and aspects
review = "Price is right and can't beat free shipping. HP ink cartridges are the best!"
aspects = ['Price']

# Extract sentiment expressions for the review and specified aspects
sentiment_expressions = extract_sentiment_expression_qa(review, aspects)
print(f"Review: '{review}' => Sentiment Expressions: {sentiment_expressions}")


Review: 'Price is right and can't beat free shipping. HP ink cartridges are the best!' => Sentiment Expressions: {'Price': 'right'}


In [46]:
# Example reviews and aspects
review = "Iâ€™ve been a faithful customer with more than one hp printer and instant ink subscription for at least five years. My hp printers constantly went offline and required a ridiculous amount of time spent each time to get them back online.  I work from home and although I enjoyed the convenience of having the instant ink delivered to my house, the constant issues with my printers, which took up a great deal of my working time more than I wanted to also once I really sat down and took the time to look at the expense of my instant subscription I think I was being ripped off. Completely not worth it."
aspects = ['Price']

# Extract sentiment expressions for the review and specified aspects
sentiment_expressions = extract_sentiment_expression_qa(review, aspects)
print(f"Review: '{review}' => Sentiment Expressions: {sentiment_expressions}")


Review: 'Iâ€™ve been a faithful customer with more than one hp printer and instant ink subscription for at least five years. My hp printers constantly went offline and required a ridiculous amount of time spent each time to get them back online.  I work from home and although I enjoyed the convenience of having the instant ink delivered to my house, the constant issues with my printers, which took up a great deal of my working time more than I wanted to also once I really sat down and took the time to look at the expense of my instant subscription I think I was being ripped off. Completely not worth it.' => Sentiment Expressions: {'Price': 'Not found'}


# T5 model - prompt engineering w/o fine-tuning

In [7]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

# Load pre-trained T5 model and tokenizer (new tokenizer behavior with legacy=False)
model = T5ForConditionalGeneration.from_pretrained('t5-small')
tokenizer = T5Tokenizer.from_pretrained('t5-small', legacy=False)

def extract_sentiment_t5(review, aspects):
    # Store the generated sentiment expressions
    sentiment_expressions = {}

    # Iterate over the provided aspects to construct prompts
    for aspect in aspects:
        # Update the prompt to be clearer and directly request sentiment extraction
        prompt = f"What is the sentiment of the aspect '{aspect}' in the following review: '{review}'?"
        
        # Tokenize the input text
        input_ids = tokenizer.encode(prompt, return_tensors='pt')
        
        # Generate the output using the T5 model
        outputs = model.generate(input_ids, max_length=50, num_beams=4, early_stopping=True)
        
        # Decode the generated response
        sentiment = tokenizer.decode(outputs[0], skip_special_tokens=True)
        
        # Store the sentiment expression
        sentiment_expressions[aspect] = sentiment
    
    return sentiment_expressions

# Example reviews and aspects
review = "The quality is good but the price is expensive."
aspects = ["quality", "price"]

# Extract sentiment expressions for the review and specified aspects
sentiment_expressions = extract_sentiment_t5(review, aspects)
print(f"Review: '{review}' => Sentiment Expressions: {sentiment_expressions}")


Review: 'The quality is good but the price is expensive.' => Sentiment Expressions: {'quality': "What is the sentiment of the aspect 'quality' in the following review: 'The quality is good but the price is expensive'?", 'price': "What is the sentiment of the aspect 'price' in the following review: 'The quality is good but the price is expensive'?"}
